In [356]:
import numpy as np
import matplotlib.pyplot as plt
from mayavi.mlab import *

plt.style.use('bmh_TGV1')
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from scipy.signal import zoom_fft as zoom_fft

import Optics

In [419]:
"RICHARDS-WOLF VECTORIAL FOCUSING"
'Following [Leutenegger2006]'

"LIGHT PARAMETERS"
lam = 488e-9
k = 2*np.pi/lam
IR = Optics.Beam(w0=2e-2, lam=lam)

"LENS PARAMETERS"
f = 10e-2
NA = 0.5
R = NA*f 

"INPUT k GRID"
Nx, Ny = 1024, 1024
Lmax = min(f, 1.5*R) # maximum coordinate in the input plane, either limited by the lens size or the aperture
kmax = k*Lmax/np.sqrt(Lmax**2+f**2) # maximum transverse wavevector
k_grid = IR.Grid(Lx=2*kmax, Ly=2*kmax, Nx=Nx, Ny=Ny)
Lkx, Lky = 2*kmax, 2*kmax
dkx, dky = Lkx/(Nx-1), Lky/(Ny-1)

kx_ax, ky_ax = k_grid[0][0], k_grid[1].T[0] # retrieve axes from meshgrid
kx, ky = k_grid
kz = np.sqrt(k**2-kx**2-ky**2)

# compute x and y axes from the regular k grid
x = -(kx/k) * f * (1+ (kx**2+ky**2)/kz**2)**(1/2)
y = -(ky/k) * f * (1+ (kx**2+ky**2)/kz**2)**(1/2)

In [420]:
'Returns the 3 input field components at a given (x,y) point of the input plane'
def Ei(x, y):

    # Smooth entrance pupil
    r = np.sqrt(x**2+y**2)
    dR = R/30
    pupil = (1/2)*(1+np.tanh(1.5*(R-r)/dR))

    # Three components
    Ex = IR.LGBeam(x, y, z=0, l=0, p=0)*pupil
    Ey = IR.LGBeam(x, y, z=0, l=0, p=0)*pupil*0
    Ez = IR.LGBeam(x, y, z=0, l=0, p=0)*0*pupil

    return [Ex, Ey, Ez]

def zoom_fft2(M, fnx, fny, fsx, fsy, mx, my):
    TFx = zoom_fft(M, fn=fnx, m=mx, fs=fsx, axis=1)
    TFxy = zoom_fft(TFx, fn=fny, m=my, fs=fsy, axis=0)
    return TFxy

def RW(Ei, z, xmin, xmax, ymin, ymax, Mx, My):

    r2 = x**2+y**2
    l = np.sqrt(r2+f**2) # optical path length from x,y to focus

    # input field
    Ex, Ey, Ez = Ei

    # Rotation of the E field at the lens
    Etx = Ex + (x/r2)*((f/l)-1)*( x*Ex + y*Ey ) -(x/l)*Ez
    Ety = Ey + (y/r2)*((f/l)-1)*( x*Ex + y*Ey ) -(y/l)*Ez
    Etz = (x/l)*Ex + (y/l)*Ey + (f/l)*Ez

    # apodization factor for energy conservation
    apo = np.sqrt(f/l)
    Et = np.array([Etx, Ety, Etz]) * apo

    # Definition of the k-space
    kz = np.sqrt(k**2-kx**2-ky**2)

    # Richards-Wolf integrand
    prefactor = -1j*f/(2*np.pi)
    factor = np.exp(1j*kz*z)/kz
    Etx, Ety, Etz = Et * prefactor * factor

    # 2D Fourier transform
    Efx = zoom_fft2(Etx, [xmin, xmax], [ymin, ymax], 2*np.pi/dkx, 2*np.pi/dky, Mx, My)
    Efx = Efx * dkx * dky
    Efy = zoom_fft2(Ety, [xmin, xmax], [ymin, ymax], 2*np.pi/dkx, 2*np.pi/dky, Mx, My)
    Efy = Efy * dkx * dky
    Efz = zoom_fft2(Etz, [xmin, xmax], [ymin, ymax], 2*np.pi/dkx, 2*np.pi/dky, Mx, My)
    Efz = Efz * dkx * dky

    field = np.array([Efx, Efy, Efz])

    # Compute the conjugate axes
    FT_x_axis = np.linspace(xmin, xmax, Mx, endpoint=False)
    FT_y_axis = np.linspace(ymin, ymax, My, endpoint=False)
    FT_grid = np.meshgrid(FT_x_axis, FT_y_axis)

    return field, FT_grid


In [421]:
xmin, xmax = -3e-6, 3e-6
ymin, ymax = -3e-6, 3e-6
Mx, My = 1024, 1024
input = Ei(x, y)

test, tgrid = RW(input, 0, xmin, xmax, ymin, ymax, Mx, My)

In [422]:
%matplotlib qt
fig, ax = plt.subplots(2, 3)
cmap='nipy_spectral'

Efx, Efy, Efz = test
u, v = tgrid[0][0], tgrid[1].T[0]
extentuv=[u[0], u[-1], v[0], v[-1]]

regular_grid = IR.Grid(Lx=2*Lmax, Ly=2*Lmax, Nx=Nx, Ny=Ny)
Eix, Eiy, Eiz = Ei(*regular_grid)
extentxy=[-Lmax, Lmax, -Lmax, Lmax]

ax[0, 0].imshow(np.abs(Eix), extent=extentxy, origin='lower', cmap=cmap)
ax[0, 1].imshow(np.abs(Eiy), extent=extentxy, origin='lower', cmap=cmap, vmax=np.max(np.abs(Eix)))
ax[0, 2].imshow(np.abs(Eiz), extent=extentxy, origin='lower', cmap=cmap, vmax=np.max(np.abs(Eix)))

ax[1, 0].imshow(np.abs(Efx), extent=extentuv, origin='lower', cmap=cmap)
ax[1, 1].imshow(np.abs(Efy), extent=extentuv, origin='lower', cmap=cmap, vmax=np.max(np.abs(Efx)))
ax[1, 2].imshow(np.abs(Efz), extent=extentuv, origin='lower', cmap=cmap, vmax=np.max(np.abs(Efx)))

In [354]:
z_grid = np.linspace(-20*lam, 20*lam, 50)

E_list=[]
for z in z_grid:
    xmin, xmax = -3e-6, 3e-6
    ymin, ymax = -3e-6, 3e-6
    Mx, My = 512, 512
    input = Ei(x, y)
    test, tgrid = RW(input, z, xmin, xmax, ymin, ymax, Mx, My)
    Ex, Ey, Ez = test
    E = np.sqrt(np.abs(Ex**2)+np.abs(Ey**2)+np.abs(Ez**2))
    E_list.append(E**2)

E_list=np.array(E_list)

In [355]:
from mayavi.mlab import *
E_list = E_list / np.max(E_list)
contourlist = [1/np.e, 1/np.e**2, 1/np.e**3, 1/np.e**4]
contour3d(E_list,contours=contourlist,opacity=.2 )